# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
# Set the PySpark environment variables
import os
# os.environ['SPARK_HOME'] = "/Users/coder2j/Apps/Spark"
os.environ['SPARK_HOME'] = "C:\spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [56]:
import pyspark

In [57]:
# Import PySpark
from pyspark.sql import SparkSession

In [3]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySpark-Get-Started") \
    .getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

### Step 3. Assign it to a variable called chipo.

In [55]:
from pyspark import SparkFiles

In [4]:
chipotle = spark.read.csv('chipotle.tsv', sep='\t', header=True, inferSchema=True)

In [ ]:
#------------------------------------------------------------------------

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ex2").getOrCreate()
spark

In [3]:
from pyspark import SparkFiles

In [4]:

url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"

spark.sparkContext.addFile(url)

chipotle = spark.read.csv(SparkFiles.get("chipotle.tsv"), header=True, inferSchema=True, sep='\t')

### Step 4. See the first 10 entries

In [5]:
chipotle.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Step 5. What is the number of observations in the dataset?

In [6]:
# Solution 1
print(chipotle.count(), ',' , len(chipotle.columns))


4622 , 5


In [23]:
# Solution 2
chipotle.createOrReplaceTempView('data')
row_count = spark.sql('SELECT COUNT(*) FROM data').collect()[0][0]
row_count

4622

### Step 6. What is the number of columns in the dataset?

In [7]:
len(chipotle.columns)

5

### Step 7. Print the name of all the columns.

In [8]:
chipotle.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### Step 8. How is the dataset indexed?

### Step 9. Which was the most-ordered item? 

In [9]:
most_ordered_item = chipotle.groupBy('item_name').sum().orderBy('sum(quantity)', ascending=0).show(1)

+------------+-------------+-------------+
|   item_name|sum(order_id)|sum(quantity)|
+------------+-------------+-------------+
|Chicken Bowl|       713926|          761|
+------------+-------------+-------------+
only showing top 1 row



### Step 10. For the most-ordered item, how many items were ordered?

In [10]:
most_ordered_item = chipotle.groupBy('item_name').sum().orderBy('sum(quantity)', ascending=0).show(1)

+------------+-------------+-------------+
|   item_name|sum(order_id)|sum(quantity)|
+------------+-------------+-------------+
|Chicken Bowl|       713926|          761|
+------------+-------------+-------------+
only showing top 1 row



### Step 11. What was the most ordered item in the choice_description column?

In [11]:
most_ordered_item = chipotle.groupBy('choice_description').sum().orderBy('sum(quantity)', ascending=0).show(2)

+------------------+-------------+-------------+
|choice_description|sum(order_id)|sum(quantity)|
+------------------+-------------+-------------+
|              NULL|      1178731|         1382|
|       [Diet Coke]|       123455|          159|
+------------------+-------------+-------------+
only showing top 2 rows



### Step 12. How many items were orderd in total?

In [12]:
total_quantity = chipotle.agg({"quantity":"sum"}).show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



### Step 13. Turn the item price into a float

In [13]:
chipotle.dtypes

[('order_id', 'int'),
 ('quantity', 'int'),
 ('item_name', 'string'),
 ('choice_description', 'string'),
 ('item_price', 'string')]

#### Step 13.a. Check the item price type

In [15]:
chipotle.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



#### Step 13.b. Create a lambda function and change the type of item price

In [6]:
from pyspark.sql.functions import regexp_replace

In [7]:
price = chipotle.withColumn("price",regexp_replace("item_price","$",""))
price.show(2)

+--------+--------+--------------------+------------------+----------+------+
|order_id|quantity|           item_name|choice_description|item_price| price|
+--------+--------+--------------------+------------------+----------+------+
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 |$2.39 |
|       1|       1|                Izze|      [Clementine]|    $3.39 |$3.39 |
+--------+--------+--------------------+------------------+----------+------+
only showing top 2 rows



In [8]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType, FloatType

In [9]:
def dolariser(x):
    return str(x[1:-1])

udf_dolariser = udf(lambda x:dolariser(x),StringType())

In [10]:
price_df = chipotle.withColumn("item_price",udf_dolariser(col("item_price")))

In [11]:
price_df.show(2)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|      2.39|
|       1|       1|                Izze|      [Clementine]|      3.39|
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows



#### Step 13.c. Check the item price type

In [12]:
price_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



In [13]:
price_df = price_df.withColumn("item_price", price_df["item_price"].cast('float'))

In [14]:
price_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)



### Step 14. How much was the revenue for the period in the dataset?

In [16]:
revenue_result = price_df.withColumn("revenue", price_df["item_price"]*price_df["quantity"])
total_revenue = revenue_result.agg({"revenue":"sum"})
total_revenue.show()

+----------------+
|    sum(revenue)|
+----------------+
|39237.0197327137|
+----------------+



In [21]:
print("Total Revenue: $",total_revenue.head()[0])

Total Revenue: $ 39237.0197327137


### Step 15. How many orders were made in the period?

In [22]:
from pyspark.sql.functions import countDistinct

In [23]:
total_orders = price_df.select(countDistinct("order_id"))

In [24]:
print("Total Orders: ",total_orders.head(1)[0][0])

Total Orders:  1834


### Step 16. What is the average revenue amount per order?

In [3]:
# Solution 1



In [4]:
# Solution 2



### Step 17. How many different items are sold?